# Определение перечня электроэнергетических режимов для последующего расчета

## Подготовка к расчетам

Импорт бибилиотек

In [1]:
import pandas as pd
import numpy as np
from itertools import *

Получение исходных данных

In [2]:
data = pd.read_excel('test.xlsx', sheet_name = 'id_group')

In [3]:
data

,id_group,group_name,unique,parall
0,1,КВЛ 110 кВ Левобережная – Кемчуг тяговая II це...,0,2
1,2,КВЛ 110 кВ Левобережная – Кемчуг тяговая I цеп...,1,0
2,3,ВЛ 110 кВ Ачинский НПЗ – Ачинск тяговая II цеп...,0,4
3,4,ВЛ 110 кВ Ачинский НПЗ – Ачинск тяговая I цепь...,1,0
4,5,ВЛ 220 кВ Назаровская ГРЭС – Ачинский НПЗ II ц...,0,6
5,6,ВЛ 220 кВ Назаровская ГРЭС – Ачинский НПЗ I це...,1,0
6,7,АТ2 220/110/10 кВ ПС 220 кВ Ачинский НПЗ,0,8
7,8,АТ1 220/110/10 кВ ПС 220 кВ Ачинский НПЗ,1,0
8,9,АТ2А/2Б 220/110 кВ Назаровская ГРЭС,1,0
9,10,ВЛ 110 кВ Ново-Анжерская – Иверка с отпайками,1,0


Значение '1' в столбце 'unique' отражает уникальность сетевого элемента.

Значение '0' в столбце 'unique' говорит о том, что элемент дублирующий.

Значения в столбце 'parall' относятся только к дублирующим элементам и отражают id_group элемента, который они дублируют.

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id_group    11 non-null     int64 
 1   group_name  11 non-null     object
 2   unique      11 non-null     int64 
 3   parall      11 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 480.0+ bytes


Создадим датафрейм с уникальными элементами

In [5]:
data_unique = data.copy()
data_unique = data_unique.loc[data_unique['unique'] == 1]

In [6]:
data_unique

,id_group,group_name,unique,parall
1,2,КВЛ 110 кВ Левобережная – Кемчуг тяговая I цеп...,1,0
3,4,ВЛ 110 кВ Ачинский НПЗ – Ачинск тяговая I цепь...,1,0
5,6,ВЛ 220 кВ Назаровская ГРЭС – Ачинский НПЗ I це...,1,0
7,8,АТ1 220/110/10 кВ ПС 220 кВ Ачинский НПЗ,1,0
8,9,АТ2А/2Б 220/110 кВ Назаровская ГРЭС,1,0
9,10,ВЛ 110 кВ Ново-Анжерская – Иверка с отпайками,1,0
10,11,ВЛ 110 кВ Ново-Анжерская – Яйская с отпайкой н...,1,0


Создадим датафрейм с дублирующими элементами

In [7]:
data_duplicate = data.copy()
data_duplicate = data_duplicate.loc[data_duplicate['unique'] == 0]
data_duplicate

,id_group,group_name,unique,parall
0,1,КВЛ 110 кВ Левобережная – Кемчуг тяговая II це...,0,2
2,3,ВЛ 110 кВ Ачинский НПЗ – Ачинск тяговая II цеп...,0,4
4,5,ВЛ 220 кВ Назаровская ГРЭС – Ачинский НПЗ II ц...,0,6
6,7,АТ2 220/110/10 кВ ПС 220 кВ Ачинский НПЗ,0,8


Создадим шаблон для отчета

In [8]:
id_regim = pd.DataFrame(columns=['id_regim', 'N_1', 'N_2', 'N_3', 'regim_name'])

## Нормальный режим

In [9]:
id_regim_N = id_regim.copy()
id_regim_N.loc[0,:] = [None, None, None, None, 'Нормальный режим']
id_regim_N

,id_regim,N_1,N_2,N_3,regim_name
0,None,None,None,None,Нормальный режим


## Режимы N-1

Количество режимов N-1 равно числу уникальных элементов

In [10]:
id_regim_N_1 = id_regim.copy()
id_regim_N_1['N_1'] = data_unique['id_group']
id_regim_N_1

,id_regim,N_1,N_2,N_3,regim_name
1,NaN,2,NaN,NaN,NaN
3,NaN,4,NaN,NaN,NaN
5,NaN,6,NaN,NaN,NaN
7,NaN,8,NaN,NaN,NaN
8,NaN,9,NaN,NaN,NaN
9,NaN,10,NaN,NaN,NaN
10,NaN,11,NaN,NaN,NaN


## Режимы N-2

Создание первой части режимов N-2 выполняется как сочетания без повторений уникальных элементов. 

In [11]:
N_2_1 = []
for s in list(combinations(data_unique['id_group'], 2)):
    N_2_1.append(list(s))
data_N_2_1 = pd.DataFrame(N_2_1, columns = ['N_1', 'N_2'])
data_N_2_1

,N_1,N_2
0,2,4
1,2,6
2,2,8
3,2,9
4,2,10
5,2,11
6,4,6
7,4,8
8,4,9
9,4,10


Вторая часть режимов N-2 выполняется как отключение дублирующего элемента в схеме ремонта соответствующего уникального элемента

In [12]:
data_N_2_2 = pd.DataFrame(columns = ['N_1', 'N_2'])
data_N_2_2['N_1'] = data_duplicate['id_group'] 
data_N_2_2['N_2'] = data_duplicate['parall']
data_N_2_2

,N_1,N_2
0,1,2
2,3,4
4,5,6
6,7,8


Объединение режимов N-2 и приведение к единой форме

In [13]:
data_N_2 = pd.concat([data_N_2_1, data_N_2_2], axis=0).reset_index(drop=True)

In [14]:
id_regim_N_2 = id_regim.copy()
id_regim_N_2['N_1'] = data_N_2['N_1']
id_regim_N_2['N_2'] = data_N_2['N_2']
id_regim_N_2

,id_regim,N_1,N_2,N_3,regim_name
0,NaN,2,4,NaN,NaN
1,NaN,2,6,NaN,NaN
2,NaN,2,8,NaN,NaN
3,NaN,2,9,NaN,NaN
4,NaN,2,10,NaN,NaN
5,NaN,2,11,NaN,NaN
6,NaN,4,6,NaN,NaN
7,NaN,4,8,NaN,NaN
8,NaN,4,9,NaN,NaN
9,NaN,4,10,NaN,NaN


## Режимы N-3

Создание первой части режимов N-3 выполняется как сочетания без повторений уникальных элементов. 

In [15]:
N_3_1 = []
for s in list(combinations(data_unique['id_group'], 3)):
    N_3_1.append(list(s))
data_N_3_1 = pd.DataFrame(N_3_1, columns = ['N_1', 'N_2', 'N_3'])

Вторая часть режимов N-3 выполняется на основе перечня N_2_2.

К N_2_2 добавляется уникальный элемент, и процесс повторяется для каждого уникального элемента.

In [16]:
data_N_3_2 = pd.DataFrame(columns = ['N_1', 'N_2', 'N_3'])
data_N_3_2['N_3'] = None
for id_group in data_unique['id_group']:
    iter_data = data_N_2_2.copy()
    iter_data['N_3'] = id_group
    data_N_3_2 = pd.concat([data_N_3_2, iter_data], axis=0).reset_index(drop=True)

Удаление дубликатов в N_3_2 (несуществующих режимов)

In [17]:
data_N_3_2 = data_N_3_2[data_N_3_2['N_2'] != data_N_3_2['N_3']]

Объединение режимов N-3 и приведение к единой форме

In [18]:
data_N_3 = pd.concat([data_N_3_1, data_N_3_2], axis=0).reset_index(drop=True)

In [19]:
id_regim_N_3 = id_regim.copy()
id_regim_N_3 = id_regim.copy()
id_regim_N_3['N_1'] = data_N_3['N_1']
id_regim_N_3['N_2'] = data_N_3['N_2']
id_regim_N_3['N_3'] = data_N_3['N_3']

## Формирование сводного перечня

Объединение всех режимов в единой форме

In [20]:
id_regim = pd.concat([id_regim_N, id_regim_N_1, id_regim_N_2, id_regim_N_3], axis=0).reset_index(drop=True)
id_regim

,id_regim,N_1,N_2,N_3,regim_name
0,None,None,None,None,Нормальный режим
1,NaN,2,NaN,NaN,NaN
2,NaN,4,NaN,NaN,NaN
3,NaN,6,NaN,NaN,NaN
4,NaN,8,NaN,NaN,NaN
...,...,...,...,...,...
87,NaN,7,8,10,NaN
88,NaN,1,2,11,NaN
89,NaN,3,4,11,NaN
90,NaN,5,6,11,NaN


Задание id_regim

In [21]:
for i in id_regim.index:
    id_regim.loc[i, 'id_regim'] = i + 1
id_regim    

,id_regim,N_1,N_2,N_3,regim_name
0,1,None,None,None,Нормальный режим
1,2,2,NaN,NaN,NaN
2,3,4,NaN,NaN,NaN
3,4,6,NaN,NaN,NaN
4,5,8,NaN,NaN,NaN
...,...,...,...,...,...
87,88,7,8,10,NaN
88,89,1,2,11,NaN
89,90,3,4,11,NaN
90,91,5,6,11,NaN


Формирование названий режимов

In [22]:
for i in id_regim.index:
    if i > 0:
        if id_regim.loc[i, 'N_2'] is np.NaN:
            id_regim.loc[i, 'regim_name'] = f"Отключение {list(data[data['id_group'] == id_regim.loc[i, 'N_1']]['group_name'])[0]}"
        elif id_regim.loc[i, 'N_3'] is np.NaN:
            id_regim.loc[i, 'regim_name'] = (f"Отключение {list(data[data['id_group'] == id_regim.loc[i, 'N_1']]['group_name'])[0]} и" 
            f" {list(data[data['id_group'] == id_regim.loc[i, 'N_2']]['group_name'])[0]}")
        else:
            id_regim.loc[i, 'regim_name'] = (f"Отключение {list(data[data['id_group'] == id_regim.loc[i, 'N_1']]['group_name'])[0]}, " 
            f"{list(data[data['id_group'] == id_regim.loc[i, 'N_2']]['group_name'])[0]} и"
            f" {list(data[data['id_group'] == id_regim.loc[i, 'N_3']]['group_name'])[0]}")

In [23]:
id_regim

,id_regim,N_1,N_2,N_3,regim_name
0,1,None,None,None,Нормальный режим
1,2,2,NaN,NaN,Отключение КВЛ 110 кВ Левобережная – Кемчуг тя...
2,3,4,NaN,NaN,Отключение ВЛ 110 кВ Ачинский НПЗ – Ачинск тяг...
3,4,6,NaN,NaN,Отключение ВЛ 220 кВ Назаровская ГРЭС – Ачинск...
4,5,8,NaN,NaN,Отключение АТ1 220/110/10 кВ ПС 220 кВ Ачински...
...,...,...,...,...,...
87,88,7,8,10,Отключение АТ2 220/110/10 кВ ПС 220 кВ Ачински...
88,89,1,2,11,Отключение КВЛ 110 кВ Левобережная – Кемчуг тя...
89,90,3,4,11,Отключение ВЛ 110 кВ Ачинский НПЗ – Ачинск тяг...
90,91,5,6,11,Отключение ВЛ 220 кВ Назаровская ГРЭС – Ачинск...


Сохранение результата

In [24]:
id_regim.to_excel('result.xlsx')